In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import random
import multiprocessing as mp
import pickle
import math
from scipy.stats import skew, kurtosis

# Functions

## States and Gaussian noise

In [2]:
def create_ranges(true_conductances, num_states):
    max_value = max(true_conductances.flatten())
    min_value =  min(true_conductances.flatten())
    ranges = [(x, y) for x, y in zip(np.linspace(min_value, max_value, num_states+1)[:-1], np.linspace(min_value, max_value, num_states+1)[1:])]
    return ranges


In [15]:
def create_state_info(num_states, ranges, true_conductances, sampled_conductances):
    states_info = {}
    for i in range(num_states):
        state_key = i
        states_info[state_key] = {
            'values': np.array([]),
            'obs': np.array([]),
            'diffs': np.array([])
        }
    
    flattened_conductances = true_conductances.flatten()
    flattened_sampled_conductances = sampled_conductances.flatten()
    starts, ends = np.array(ranges).T
    max_value = np.max(ends)
    min_value = np.min(starts)

    for t in range(np.size(sampled_conductances)): # for t in TxMC
        if flattened_conductances[t] == min_value:
            state_at_t = 0
        elif flattened_conductances[t] == max_value:
            state_at_t = num_states-1
        else:
            state_at_t = np.where((starts <= flattened_conductances[t]) & (flattened_conductances[t] < ends))[0][0]
        states_info[state_at_t]['diffs'] = np.append( states_info[state_at_t]['diffs'] , flattened_sampled_conductances[t] - flattened_conductances[t])
        states_info[state_at_t]['obs'] = np.append( states_info[state_at_t]['obs'] , flattened_sampled_conductances[t])
        states_info[state_at_t]['values'] = np.append( states_info[state_at_t]['values'] , flattened_conductances[t])
    return states_info


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, shapiro, kstest

def ecdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n+1) / n
    return x, y

def prove_gaussian_noise(num_states, states_info, scenario, measure, ratio):
    means = []
    stds = []
    p_values = np.zeros(num_states)
    states = []

    num_cols = int(np.ceil(np.sqrt(num_states)))
    num_rows = int(np.ceil(num_states / num_cols))
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
    axes = axes.flatten() 

    for i in range(num_states):
        data = states_info[i]['obs']
        mean = np.mean(data)
        std_dev = np.std(data)
        x_ecdf, y_ecdf = ecdf(data)
        
        standarized_data = (data - mean) / std_dev
        stat, p_value = shapiro(standarized_data)
        stat_ks, p_value_ks = kstest(standarized_data, 'norm')
        p_values[i] = p_value

        x = np.linspace(mean - 3 * std_dev, mean + 3 * std_dev, 1000)
        cdf = norm.cdf(x, loc=mean, scale=std_dev)
        
        states.append(np.mean(states_info[i]['values']))
        means.append(np.mean(states_info[i]['obs']))
        stds.append(np.std(states_info[i]['obs']))

        #print(measure,",", scenario,",", ratio, ",", i, ",",p_value_ks,",",  p_value>0.05)

        axes[i].plot(x_ecdf, y_ecdf, marker='.', linestyle='none', color='blue', label='ECDF')
        axes[i].plot(x, cdf, color='red', label='Theoretical Normal CDF')
        axes[i].set_xlabel('Data')
        axes[i].set_ylabel(f'shap p-value: {p_value:.3f}')
        axes[i].set_title(f'State {i+1} ks p: {round(p_value_ks,3)}')
        axes[i].grid(True)
        axes[i].legend()

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    plt.suptitle(f'cdf {measure} ratio {ratio}')
    plt.tight_layout()
    #fig.savefig(f'{scenario}/cdfs_{measure}/cdfs_ratio{int(ratio*10)}/cdf2_{num_states}_states', dpi=300) 
    #plt.show()
    plt.close(fig)

    return means, stds, states 

## Model parameters

In [17]:
import numpy as np

def trainP(num_states, ranges, true_conductances):
    n = num_states
    max_value = np.max(true_conductances)
    min_value = np.min(true_conductances)
    # Sample data: sequences of observed states
    data = np.zeros((true_conductances.shape[0],true_conductances.shape[1]))
    
    starts, ends = np.array(ranges).T
    for i in range(true_conductances.shape[0]):
        for j in range(true_conductances.shape[1]):
            if true_conductances[i,j] == max_value:
                data[i, j] = 0
            elif true_conductances[i,j] == min_value:
                data[i, j] = num_states-1
            else:
                data[i, j] = np.where((starts <= true_conductances[i,j]) & (true_conductances[i,j] < ends))[0][0]
    P = np.zeros((n, n))
    initial_state_count = np.zeros(n)
    
    # Count transitions
    for sequence in data:
        initial_state = int(sequence[0])
        initial_state_count[initial_state] += 1
        for i in range(len(sequence) - 1):
            current_state = int(sequence[i])
            next_state = int(sequence[i + 1])
            P[current_state, next_state] += 1
    
    # Convert counts to probabilities
    pi0 = initial_state_count / initial_state_count.sum()
    for i in range(n):
        P[i, :] /= P[i, :].sum()

    return P, pi0

    

In [18]:
def MBtransition(A, T, t, k):
    m = len(A)
    n = len(A[0])
    B = np.zeros((m, n))
    A1 = np.linalg.matrix_power(A, T - t - 1)
    A2 = np.linalg.matrix_power(A, T - t)
    if t==T-1:
        return np.eye(m)
    for i in range(m):
        for j in range(n):
            if A1[j, k]==0 or A[i, j]==0:
                B[i, j] = 0
            else:
                B[i, j] = A[i, j] * A1[j, k] / A2[i, k]
    return B

## Filters

In [19]:
from scipy.stats import norm

def hmm_filter(P, pi0, cond, cond_sampled, states, gaussian_parameters):
    q_hmm = []
    t = 0
    state_estimates_hmm = []
    
    O = np.diag([ norm.pdf(cond_sampled[0],gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
    
    numerator = O @ P.T @ (pi0).T
    denominator = np.sum(O @ P.T @ (pi0).T)
    qt = numerator / denominator
    q_hmm.append(qt)
    real_state = cond[t]
    estimated_state = states[np.argmax(qt)]
    state_estimates_hmm.append(estimated_state)

    error_hmm = np.zeros(T)
    error_hmm[0] = (estimated_state-real_state)
    #print("t: ",t, "obs:", cond_sampled[0], "estimated", np.round(estimated_state,3), "real one", real_state, "error:", error_hmm[t]**2)
        
    t +=1
    
    for obs in cond_sampled[1:]:
        #O = np.diag([ norm.pdf(obs-states[i], gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
        O = np.diag([ norm.pdf(obs, gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
        numerator = O @ P.T @ q_hmm[t-1]
        denominator = np.sum(O @ P.T @ q_hmm[t-1])
        qt = numerator / denominator
        
        estimated_state = states[np.argmax(qt)]
        real_state = cond[t]
        error_hmm[t] = (estimated_state-real_state) 
    
        q_hmm.append(qt)
        state_estimates_hmm.append(estimated_state)
        #print("t: ",t, "obs:", obs, "estimated", np.round(estimated_state,3), "real one", real_state, "error:", error_hmm[t]**2)
        t+=1
    
    hmm_squared_errors = error_hmm** 2
    return hmm_squared_errors, q_hmm


In [20]:
from scipy.stats import norm

def hmb_filter(Bt, pi0, cond, cond_sampled, states,gaussian_parameters):
    q_hmb = []
    t = 0
    state_estimates_hmb = []
    
    O = np.diag([ norm.pdf(cond_sampled[0],gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
    numerator = O @ (Bt[t]).T @ (pi0).T
    denominator = np.sum(O @ (Bt[t]).T @ (pi0 ).T)
    qt = numerator / denominator
    q_hmb.append(qt)
    real_state = cond[t]
    estimated_state = states[np.argmax(qt)]
    state_estimates_hmb.append(estimated_state)

    error_hmb = np.zeros(T)
    error_hmb[0] = (estimated_state-real_state)
    #print("t: ",t, "obs:", cond_sampled[0], "estimated", np.round(estimated_state,3), "real one", real_state, "error:", error_hmb[t]**2)
        
    t +=1
    for obs in cond_sampled[1:]:
        #O = np.diag([ norm.pdf(obs-states[i], gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
        O = np.diag([ norm.pdf(obs, gaussian_parameters[i, 0], gaussian_parameters[i, 1]) for i in range(num_states)] )
        numerator = O @ (Bt[t]).T @ q_hmb[t-1]
        denominator = np.sum(O @ (Bt[t]).T @ q_hmb[t-1])
        qt = numerator / denominator
    
        """
        weight_average = qt * states
        idx = np.argmin(np.abs(weight_average - np.sum(states)))
        estimated_state = states[idx]
        """
        estimated_state = states[np.argmax(qt)]
        real_state = cond[t]
        error_hmb[t] = (estimated_state-real_state)
        
        q_hmb.append(qt)
        state_estimates_hmb.append(estimated_state)
        
        #print("t: ",t, "obs:", obs, "estimated", np.round(estimated_state,3), "real one", real_state, "error:", error_hmb[t]**2)
        t+=1
    
    hmb_squared_errors = error_hmb ** 2
    return hmb_squared_errors, q_hmb

In [21]:
def write_x_to_txt(xs, filename):
    with open(filename, 'w') as file:
        for x in xs:
            file.write(f"{x}\n")

# Simulation and measures

In [22]:
import networkx as nx

def polarization_score_sampled(G, ratio):
  num_edges = G.number_of_edges()
  sample_size = math.floor(ratio*num_edges)

  # Sampling the edges and copying the nodes attributes
  random_sample_edges = random.sample(list(G.edges), sample_size)
  G_sample = nx.Graph()
  G_sample.add_edges_from(random_sample_edges)
  for node in G_sample.nodes():
    G_sample.nodes[node]['opinion'] = G.nodes[node]['opinion']
    
  intraideological_conn = 0
  cross_ideological_conn = 0
  for u, v in G_sample.edges():
    if G_sample.nodes[u]['opinion'] * G_sample.nodes[v]['opinion'] > 0:
      intraideological_conn += 1
    else: 
      cross_ideological_conn +=1
      #print("u: ", G.nodes[u]['opinion'], "v: ",G.nodes[v]['opinion'])
  pol_score = abs(intraideological_conn-cross_ideological_conn) / (intraideological_conn+cross_ideological_conn)  
  
  return pol_score

def polarization_score(G):
  intraideological_conn = 0
  cross_ideological_conn = 0
  for u, v in G.edges():
    if G.nodes[u]['opinion']*G.nodes[v]['opinion'] >0:
      intraideological_conn += 1
    else: 
      cross_ideological_conn +=1
      #print("u: ", G.nodes[u]['opinion'], "v: ",G.nodes[v]['opinion'])
  pol_score = abs(intraideological_conn-cross_ideological_conn) / (intraideological_conn+cross_ideological_conn)  
  
  return pol_score

def algebraic_connectivity(graph):
  conn = nx.algebraic_connectivity(graph, weight ='weight')
  return conn

def sampled_algebraic_connectivity(graph, ratio):
  num_edges = graph.number_of_edges()
  sample_size = math.floor(ratio*num_edges)

  random_sample_edges = random.sample(list(graph.edges), sample_size)
  G_sample = nx.Graph()
  G_sample.add_edges_from(random_sample_edges)
  #print("Sampled graph: ", G_sample)
  conn = nx.algebraic_connectivity(G_sample, weight ='weight')

  return conn

def bimodality_coefficient(graph):
    opinions = np.array([graph.nodes[node]['opinion'] for node in graph.nodes()])
    mean_opinion = np.mean(opinions)
    skewness = ((opinions - mean_opinion)**3).mean() / ((opinions - mean_opinion)**2).mean()**(3/2)
    kurtosis = ((opinions - mean_opinion)**4).mean() / ((opinions - mean_opinion)**2).mean()**2
    bimodality_coeff = (skewness**2 + 1) / kurtosis
    return bimodality_coeff

def sampled_bimodality_coefficient(graph, ratio):
    num_edges = graph.number_of_edges()
    sample_size = math.floor(ratio*num_edges)
    
    random_sample_edges = random.sample(list(graph.edges), sample_size)
    G_sample = nx.Graph()
    G_sample.add_edges_from(random_sample_edges)
    for node in G_sample.nodes():
        G_sample.nodes[node]['opinion'] = graph.nodes[node]['opinion']
    
    opinions = np.array([G_sample.nodes[node]['opinion'] for node in G_sample.nodes()])
    mean_opinion = np.mean(opinions)
    skewness = ((opinions - mean_opinion)**3).mean() / ((opinions - mean_opinion)**2).mean()**(3/2)
    kurtosis = ((opinions - mean_opinion)**4).mean() / ((opinions - mean_opinion)**2).mean()**2
    bimodality_coeff = (skewness**2 + 1) / kurtosis
    return bimodality_coeff

def balance(graph):
  c1 = 0
  c2 = 0

  for node in list(graph.nodes()):
    if graph.nodes[node]['opinion'] < 0:
      c1 += 1
    else: 
      c2 +=1
      #print("u: ", G.nodes[u]['opinion'], "v: ",G.nodes[v]['opinion'])
  pol_score = min(c1,c2) / max(c1,c2)
  
  return pol_score

def sampled_balance(graph,ratio):
    num_edges = graph.number_of_edges()
    sample_size = math.floor(ratio*num_edges)

    # Sampling the edges and copying the nodes attributes
    random_sample_edges = random.sample(list(graph.edges), sample_size)
    G_sample = nx.Graph()
    G_sample.add_edges_from(random_sample_edges)
    for node in G_sample.nodes():
        G_sample.nodes[node]['opinion'] = graph.nodes[node]['opinion']
    c1 = 0
    c2 = 0

    for node in list(G_sample.nodes()):
        if G_sample.nodes[node]['opinion'] < 0:
          c1 += 1
        else: 
          c2 +=1
          #print("u: ", G.nodes[u]['opinion'], "v: ",G.nodes[v]['opinion'])
    pol_score = min(c1,c2) / max(c1,c2)
    return pol_score


In [23]:
import networkx as nx
import numpy as np
import random

def initialize_agents(G, d1):
    # Initialize agent attributes within the graph
    nx.set_node_attributes(G, {i: {'opinion': random.uniform(-1, 1),
                                   'state': 'Uninformed',
                                   'exposure': 0,
                                   'hostility': 0,
                                   'd1': random.uniform(d1[0], d1[1]),
                                   'd2': 0.1,
                                   'mu': 0.1} for i in G.nodes()})

def propagate_opinions(G, phi, pd_function, pt_function, prewire_function, num_rewire):
    nodes = list(G.nodes())
    random.shuffle(nodes)  # Randomize node order to simulate asynchronous updating

    edges_to_remove = []
    edges_to_add = []

    for node in nodes:
        h = f_sentiment_both_sided_extreme()  # Assume this generates a new post sentiment
        if np.random.random() < pt_function(abs(h - G.nodes[node]['opinion'])):
            for neighbor in list(G.neighbors(node)):
                opinion_diff = abs(G.nodes[node]['opinion'] - G.nodes[neighbor]['opinion'])
                if np.random.random() < pd_function(opinion_diff, phi):
                    attraction_prob = 1 - abs(h- G.nodes[neighbor]['opinion'])
                    # Attraction or repulsion decision
                    if np.random.random() < attraction_prob:
                        G.nodes[neighbor]['opinion'] += np.sign(G.nodes[node]['opinion'] - G.nodes[neighbor]['opinion']) * 0.1
                        if G.nodes[neighbor]['opinion'] >1:
                            G.nodes[neighbor]['opinion'] =1
                        elif G.nodes[neighbor]['opinion'] <-1:
                            G.nodes[neighbor]['opinion']  =-1
                    else:
                        G.nodes[neighbor]['opinion'] -= np.sign(G.nodes[node]['opinion'] - G.nodes[neighbor]['opinion']) * 0.1
                        if G.nodes[neighbor]['opinion'] >1:
                            G.nodes[neighbor]['opinion'] =1
                        elif G.nodes[neighbor]['opinion'] <-1:
                            G.nodes[neighbor]['opinion']  =-1
                        if opinion_diff > 0.5 and np.random.random() < prewire_function(opinion_diff):
                            edges_to_remove.append((node, neighbor))

    # Apply all rewiring and edge modifications after all nodes have been processed
    
    for (node, neighbor) in edges_to_remove:
        if G.has_edge(node, neighbor):
            G.remove_edge(node, neighbor)
            
            # Collect potential new neighbors from the neighbors of the node's neighbors
            potential_new_neighbors = set()
            for n in G.neighbors(node):
                if n != neighbor:  # Exclude the neighbor being removed
                    potential_new_neighbors.update(G.neighbors(n))
            
            # Remove the current node and its existing neighbors from the potential new neighbors
            potential_new_neighbors.discard(node)
            potential_new_neighbors.difference_update(set(G.neighbors(node)))
            
            # Convert to list to use random.choice
            potential_new_neighbors = list(potential_new_neighbors)
            
            if potential_new_neighbors:
                for i in range( int(np.random.uniform(0,num_rewire))):
                    new_neighbor = random.choice(potential_new_neighbors)
                    edges_to_add.append((node, new_neighbor))
                    
    #print(edges_to_add)
    for (node, new_neighbor) in edges_to_add:
        if not G.has_edge(node, new_neighbor):
            G.add_edge(node, new_neighbor)
    

def simulate_network(N, m, T, phi, pd_function, pt_function, prewire_function, num_rewire):
    G = nx.barabasi_albert_graph(N, m)
    initialize_agents(G, [0.1, 1.0])  # Assuming some range for d1
    results = [G.copy()]
    for _ in range(T-1):
        propagate_opinions(G, phi,pd_function, pt_function, prewire_function, num_rewire)
        results.append(G.copy())
    return results

# Define probability functions
def Pt_pol(x):
    return np.cos(x * np.pi / 2) ** 2
def Pt_sim(x):
    return np.cos(x * np.pi / 2) ** 2 if x <= 1 else 0
def Pt_uni(x):
    return 1
def Pt_all(x):
    random = np.random.uniform(0,1)
    if random <0.333:
        return Pt_pol(x)
    elif random <0.666:
        return Pt_sim(x)
    else:
        return Pt_uni(x)
    
def PdI(y, phi):
    return np.cos(y * np.pi / 2 + phi) ** 2
def PdII(y, phi):
    return np.cos(y * np.pi / 4 + phi) ** 2
def PdIII(y, phi):
    return 1
    
def Prewire(y):
    return np.cos(y * np.pi / 2) ** 2 if y > 1 else 0

def f_sentiment_both_sided_extreme():
    coin = random.uniform(0, 1)
    if coin > 0.5:
        return random.uniform(0.9, 1)
    else:
        return  random.uniform(-1, -0.9)

def f_sentiment_random():
    return random.uniform(-1, 1)

# Get train and test data

In [ ]:
def worker2(N, m, T, phi, ratio, Pd,  Pt, Prewire, num_rewire):
    graphs = simulate_network(N, m, T, phi, Pd,  Pt, Prewire, num_rewire)
    true_conductances = [polarization_score(g) for g in graphs]
    samp_conductances = [polarization_score_sampled(g, ratio) for g in graphs]
    return (true_conductances, samp_conductances)
def simulate_rewiring_wrapper2(args):
    return worker2(*args)

# -------------------------------------
# PARAMETERS
# Change the measure in worker function too
"""
polarization_score
polarization_score_sampled

bimodality_coefficient
sampled_bimodality_coefficient
"""

measures = ['pol_scores', 'bc']
measure = 'pol_scores'
measure_used = 'Homophily' # Title

pol_measure = polarization_score
sampled_pol_measure = polarization_score_sampled
num_simulations = 10
MC = 200
# -------------------------------------

In [ ]:
# Define titles for each subplot

"""
I-   PdI ; Pt_pol,and phi= 0:49,
II-  PdI ; Pt_uni, and phi= 2.65,
III- PdII, PtUni, and phi= 0
IV-  PdII; Ptuni, and phi= 1:47,
V-   PdI ; Ptpol, and phi= 0, 
-VI- PdI ; Pt_sim, and phi= 1:47, 
VII- PdII; Ptsim, and phi= 0
VIII-PdII; Ptsim, and phi= 1.47
"""


titles = ['I- PdI ; Pt_pol , phi=0:49,]',
          'II- PdI ; Ptuni , and phi= 2.65,',
          'III- PdII, PtUni, phi=0',
          'IV- PdII;Ptuni, and phi 1:47,',
          'V- PdI ;Ptpol, and phi=0',
          'VI- PdI ;Ptsim, and phi= 1:47, ',
          'VII- PdII;Ptsim, and phi 0',
          'VIII- PdII;Ptall, and phi= 1.47'
         ]


N=500
MC = 200
m = 9
T = 30

prewire_function = lambda y: Prewire(y)  # Standard rewiring based on strong disagreement
num_rewire = 3

ratios = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]
scenarios = ['scenario_I',
             'scenario_II',
             'scenario_III',
             'scenario_IV',
             'scenario_V',
             'scenario_VI',
             'scenario_VII',
             'scenario_VIII']


Pd_functions = [PdI, PdI, PdII, PdII, PdI, PdI, PdII, PdII]
Pt_functions = [Pt_pol,Pt_uni,Pt_uni,Pt_uni, Pt_pol,Pt_sim,Pt_sim, Pt_sim]
phis = [0.49, 2.65, 0, 1.47, 0, 1.47, 0, 1.47]


print("Measure: ", measure)
for ratio in np.flip(ratios):
    print(f"----------------- Processing ratio {ratio}----------------- ")
    for i, scenario in enumerate(scenarios):
        # Generate opinions for the current scenario
        scenario = scenarios[i]
        print(f"Processing scenario {i+1}")
        Pd = Pd_functions[i]
        Pt = Pt_functions[i]
        phi = phis[i]

        args_list = [(N, m, T, phi, ratio, Pd,  Pt, Prewire, num_rewire) for _ in range(MC)]
        with mp.Pool() as pool:
            results = pool.map(simulate_rewiring_wrapper2, args_list)
        list_of_tuples = results
        true_conductances_test, sampled_conductances_test = zip(*results)

        # Writing training data
        print("training data")
        write_x_to_txt(true_conductances.flatten(), f'{scenario}/{measure}_ratio{int(ratio*10)}.dat')
        write_x_to_txt(sampled_conductances.flatten(), f'{scenario}/{measure}_sampled_ratio{int(ratio*10)}.dat')
        
        # saving plot of training data
        n_rows, n_cols = true_conductances.shape
        plt.figure(figsize=(10, 6))
        offset = 0.1
        for col in range(n_cols):
            x1 = np.full(n_rows, col + 1 - offset)  # Slightly left
            y1 = true_conductances[:, col]
            plt.scatter(x1, y1, color='red', alpha=0.6, label='true_conductances' if col == 0 else "")

            x2 = np.full(n_rows, col + 1 + offset)  # Slightly right
            y2 = sampled_conductances[:, col]
            plt.scatter(x2, y2, color='grey', alpha=0.6, label='sampled_conductances' if col == 0 else "")
        plt.xlabel('t')
        plt.title('Scatter Plot of Sampled vs True conductances')
        plt.xticks(range(1, n_cols + 1))  # Set x-ticks to match column numbers
        plt.yticks(np.linspace(0,1,11)) 
        plt.legend(fontsize=14)
        plt.savefig(f'{scenario}/scatter {measure} plot_ratio{int(ratio*10)}', dpi=300) 
        plt.clf()

        meaned_conductances = np.mean(true_conductances, axis=0)
        meaned_sampled_conductances = np.mean(sampled_conductances, axis=0)
        # Plot graph conductances
        t = np.linspace(0,T,len(meaned_conductances))
        plt.plot(t, meaned_conductances, label='Real conductances')
        plt.plot(t, meaned_sampled_conductances, label='Sampled conductances')
        title = f"Graph conduc with {measure} N={N} nodes and ratio {ratio}"
        plt.title(title)
        plt.xlabel("Simulation Iteration")
        plt.ylabel("Polarization Score")
        plt.legend(fontsize=14)
        #plt.show()
        plt.savefig(f'{scenario}/mean {measure} trajectory ratio{int(ratio*10)}', dpi=300) 
        plt.clf()

        # Test Data
        graphs = simulate_network(N, m, T, phi, Pd, Pt, Prewire, num_rewire)
        cond = np.array([pol_measure(g) for g in graphs])
        cond_sampled= np.array([sampled_pol_measure(g, ratio) for g in graphs])

        write_x_to_txt(cond_sampled.flatten(), f'{scenario}/test_{measure}_sampled_test_ratio{int(ratio*10)}.dat')
        write_x_to_txt(cond.flatten(), f'{scenario}/test_{measure}_test_ratio{int(ratio*10)}.dat')

        plt.plot(cond, label="true")
        plt.plot(cond_sampled, label="sampled")
        for i in range(T):
            plt.axvline(x=i, color='gray', linestyle='--', linewidth=0.5)  # Adjust color, linestyle, and linewidth as needed
        
        plt.legend(fontsize=14)
        plt.title(f'{scenario} test data {measure} ratio {ratio}')
        plt.savefig(f'{scenario}/test_{measure}_ratio{int(ratio*10)}', dpi=300) 
        #plt.show()
        plt.clf()
               

# Compare per scenario multiple tests each

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#------------------------------------------------------------
measure = 'pol_scores'
measure_used = 'Homophily' # Title
#------------------------------------------------------------
# Fixed parameters
N = 500
m = 9
T = 30
num_rewire = 3
#------------------------------------------------------------

scenarios = [
    'scenario_I', 'scenario_II', 'scenario_III', 'scenario_IV',
    'scenario_V', 'scenario_VI', 'scenario_VII', 'scenario_VIII'
]
titles = [
    'Scenario I',
    'Scenario II',
    'Scenario III',
    'Scenario IV',
    'Scenario V',
    'Scenario VI',
    'Scenario VII',
    'Scenario VIII'
]
Pd_functions = [PdI, PdI, PdII, PdII, PdI, PdI, PdII, PdII]
Pt_functions = [Pt_pol, Pt_uni, Pt_uni, Pt_uni, Pt_pol, Pt_sim, Pt_sim, Pt_all]
phis = [0.49, 2.65, 0, 1.47, 0, 1.47, 0, 1.47]
ratios = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]

#------------------------------------------------------------

print(measure)
for i, scenario in enumerate(scenarios):
    print("-------------- scenario: ", scenario, "--------------")
    num_plots = len(ratios)
    num_cols = 2
    num_rows = (num_plots + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 16))
    axes = axes.flatten()
    for index, ratio in enumerate(ratios):
        mse_hmbs = []
        mse_hmms = []
        print(ratio)
        with open(f'{scenario}/{measure}_ratio{int(ratio*10)}.dat', 'r') as input_file:
            lines = input_file.readlines()
        read_true_cond = np.array([line.strip().split() for line in lines]).astype(float)
        true_conductances = (read_true_cond[:]).reshape(MC, T)
        
        with open(f'{scenario}/{measure}_sampled_ratio{int(ratio*10)}.dat', 'r') as input_file:
            lines = input_file.readlines()
        read_sampl_cond = np.array([line.strip().split() for line in lines]).astype(float)
        sampled_conductances = (read_sampl_cond[:]).reshape(MC, T)
        
        for sim in range(num_simulations):
            print("sim", sim)
            with open(f'{scenario}/test_{measure}_test{sim}_ratio{int(ratio*10)}.dat', 'r') as input_file:
                lines = input_file.readlines()
            cond = np.array([line.strip().split() for line in lines]).astype(float).flatten()

            with open(f'{scenario}/test_{measure}_sampled_test{sim}_ratio{int(ratio*10)}.dat', 'r') as input_file:
                lines = input_file.readlines()
            cond_sampled = np.array([line.strip().split() for line in lines]).astype(float).flatten()

            min_state = 4
            max_state = 15

            mse_hmbs2 = []
            mse_hmms2 = []

            for num_states in range(min_state, max_state + 1):
                ranges = create_ranges(true_conductances, num_states)
                state_info = create_state_info(num_states, ranges, true_conductances, sampled_conductances)
                is_any_empty = any(len(state_info[state]['values']) < 3 for state in state_info)
                if not is_any_empty:
                    means, stds, states = prove_gaussian_noise(num_states, state_info, scenario, measure, ratio)
                    P, pi0 = trainP(num_states, ranges, true_conductances)
                    gaussian_parameters = np.column_stack((means, stds))

                    final_state = (np.abs(states - np.mean(true_conductances[:, -1]))).argmin()

                    Bt = [MBtransition(P, T, t, final_state) for t in range(T)]
                    hmm_squared_errors, q_hmm = hmm_filter(P, pi0, cond, cond_sampled, states, gaussian_parameters)
                    hmb_squared_errors, q_hmb = hmb_filter(Bt, pi0, cond, cond_sampled, states, gaussian_parameters)

                    mse_hmb2 = np.mean(hmb_squared_errors)
                    mse_hmm2 = np.mean(hmm_squared_errors)
                    mse_hmbs2.append(mse_hmb2)
                    mse_hmms2.append(mse_hmm2)
                else:
                    break

            mse_hmbs.append(mse_hmbs2)
            mse_hmms.append(mse_hmms2)

        mean_mse_hmb = np.mean(mse_hmbs, axis=0)
        mean_mse_hmm = np.mean(mse_hmms, axis=0)

        ax = axes[index]
        states_studied = np.linspace(min_state, max_state, max_state - min_state + 1)[0:len(mean_mse_hmb)]
        ax.plot(states_studied, mean_mse_hmb, label="HMB")
        ax.plot(states_studied, mean_mse_hmm, label="HMM")
        ax.set_ylabel("MSE", fontsize=16)
        ax.set_title(f'MSE {titles[i]} ratio {ratio}', fontsize=23)
        ax.tick_params(axis='both', which='major', labelsize=16)
        ax.legend(fontsize=17)

    plt.suptitle(f'{measure_used} Mean Squared Error {titles[i]} per ratio', fontsize=30)
    plt.savefig(f'{scenario}_MSE_overview_{measure}.png', dpi=300)
    plt.show()